# Main notebook for battery state estimation

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback


IS_COLAB = False

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/battery-state-estimation/battery-state-estimation/"
else:
    data_path = "../../"

sys.path.append(data_path)
from data_processing.lg_dataset import LgData

### Config logging

In [ ]:
reload(logging)
logging.basicConfig(format='%(asctime)s [%(levelname)s]: %(message)s', level=logging.DEBUG, datefmt='%Y/%m/%d %H:%M:%S')

# Load Data

In [ ]:
train_names = [
    '0degC/589_Mixed1',
    '0degC/589_Mixed2',
    '0degC/590_Mixed4',
    '0degC/590_Mixed5',
    '0degC/590_Mixed6',
    '0degC/590_Mixed8',

    '10degC/567_Mixed1',
    '10degC/567_Mixed2',
    '10degC/571_Mixed4',
    '10degC/571_Mixed5',
    '10degC/571_Mixed6',
    '10degC/571_Mixed8',
    
    '25degC/551_Mixed1', 
    '25degC/551_Mixed2', 
    #'25degC/552_Mixed3',
    '25degC/552_Mixed4', 
    '25degC/552_Mixed5', 
    '25degC/552_Mixed6', 
    '25degC/552_Mixed8',   
    ]
test_names = [
    '0degC/589_LA92',
    '0degC/589_UDDS',
    '0degC/589_US06',
    '0degC/590_Mixed7',    
        
    '10degC/582_LA92',
    '10degC/576_UDDS',
    '10degC/567_US06',
    '10degC/571_Mixed7',

    '25degC/551_LA92', 
    '25degC/551_UDDS', 
    '25degC/551_US06',
    '25degC/552_Mixed7', 
    ]

steps = 500

lg_data = LgData(data_path)
cycles = lg_data.get_discharge_whole_cycle(train_names, test_names, output_capacity=False, scale_test=True)
train_x, train_y, test_x, test_y = lg_data.get_discharge_multiple_step(cycles, steps)

train_y = lg_data.keep_only_y_end(train_y, steps)
test_y = lg_data.keep_only_y_end(test_y, steps)

# Model training

In [ ]:
EXPERIMENT = "lstm_soc_percentage_lg_positive_temp_500_steps_mixed_cycle_test"

experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
print(experiment_name)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    
# Model definition
opt = tf.keras.optimizers.Adam(lr=0.00001)

model = Sequential()
model.add(LSTM(256, activation='selu',
                return_sequences=True,
                input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(LSTM(256, activation='selu', return_sequences=False))
model.add(Dense(256, activation='selu'))
model.add(Dense(128, activation='selu'))
model.add(Dense(1, activation='linear'))
model.summary()

model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint(data_path + 'results/trained_model/%s_best.h5' % experiment_name, 
                             save_best_only=True, 
                             monitor='val_loss')

In [ ]:
history = model.fit(train_x, train_y, 
                                epochs=1000, 
                                batch_size=32, 
                                verbose=2,
                                validation_split=0.2,
                                callbacks = [es, mc]
                               )

In [ ]:
model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

hist_df = pd.DataFrame(history.history)
hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

### Testing

In [ ]:
results = model.evaluate(test_x, test_y)
print(results)

# Data Visualization

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=history.history['loss'],
#                     mode='lines', name='train'))
# fig.add_trace(go.Scatter(y=history.history['val_loss'],
#                     mode='lines', name='validation'))
# fig.update_layout(title='Loss trend',
#                   xaxis_title='epoch',
#                   yaxis_title='loss')
# fig.show()

In [ ]:
# train_predictions = model.predict(train_x)

In [ ]:
# cycle_num = 0
# steps_num = 8000
# step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=step_index, y=train_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
#                     mode='lines', name='SoC predicted'))
# fig.add_trace(go.Scatter(x=step_index, y=train_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
#                     mode='lines', name='SoC actual'))
# fig.update_layout(title='Results on training',
#                   xaxis_title='Step',
#                   yaxis_title='SoC percentage')
# fig.show()

In [ ]:
# test_predictions = model.predict(test_x)

In [ ]:
# cycle_num = 0
# steps_num = 8000
# step_index = np.arange(cycle_num*steps_num, (cycle_num+1)*steps_num)

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=step_index, y=test_predictions.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
#                     mode='lines', name='SoC predicted'))
# fig.add_trace(go.Scatter(x=step_index, y=test_y.flatten()[cycle_num*steps_num:(cycle_num+1)*steps_num],
#                     mode='lines', name='SoC actual'))
# fig.update_layout(title='Results on testing',
#                   xaxis_title='Step',
#                   yaxis_title='SoC percentage')
# fig.show()